In [31]:
from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense, hamiltonian_big_dense
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse, hamiltonian_big_sparse
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.models.sparse.support_ham import build_V, build_V_in_p,  basis_m_to_p_matrix_conversion , build_V_non_zero 
import quant_rotor.models.dense.support_ham as dn
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import traceback
import time
from datetime import datetime

Stress testing the hamiltonian generator.

In [2]:
# ---------- CONFIG ----------
max_state = 11   # must be odd; try increasing this cautiously
max_site = 61     # must be odd; try up to your system's memory
g = 1            # coupling strength

log_file = "hamiltonian_stress_test_log.txt"

# ---------- Logging ----------
def log(msg):
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_file, "a") as f:
        f.write(f"{timestamp} {msg}\n")

# ---------- Header ----------
log("\n===================== NEW STRESS TEST =====================")
log(f"Testing odd state/site values up to state={max_state}, site={max_site}\n")

last_success = None
start_time = time.time()

try:
    for site in range(3, max_site + 1, 2):  # only odd sites
        for state in range(3, max_state + 1, 2):  # only odd states
            try:
                log(f"Trying state={state}, site={site}...")
                t0 = time.time()
                H, _, _ = hamiltonian_sparse(state, site, g)
                t1 = time.time()
                dim = H.shape[0]
                nnz = H.nnz if sp.issparse(H) else np.count_nonzero(H)
                log(f"✓ Success: dim={dim}, nnz={nnz}, time={t1 - t0:.2f}s")
                last_success = (state, site, dim, nnz)
            except MemoryError:
                log(f"🧨 MemoryError at state={state}, site={site}. Halting test.")
                raise
            except Exception as e:
                log(f"⚠️ Exception at state={state}, site={site}")
                log(traceback.format_exc())
                raise
except Exception:
    pass

end_time = time.time()
log(f"\n⏱️ Test completed in {end_time - start_time:.2f} seconds.")

if last_success:
    s, L, d, n = last_success
    log(f"\n✅ Last successful configuration:")
    log(f"  state = {s}")
    log(f"  site  = {L}")
    log(f"  dim   = {d}")
    log(f"  nnz   = {n}")
else:
    log("\n❌ No successful Hamiltonians generated.")

KeyboardInterrupt: 

# Testing the hamiltonian big and general by comparing sparse to dense.

In [ ]:
from scipy.sparse.linalg import norm

def sparse_allclose(A, B, atol=1e-8, rtol=1e-5):
    if A.shape != B.shape:
        return False
    diff = A - B
    return norm(diff, ord='fro') <= atol + rtol * max(norm(A, ord='fro'), norm(B, ord='fro'))

In [ ]:
def compare_ground_states(H_dense, H_sparse, atol=1e-6):
    from scipy.sparse.linalg import eigsh
    eigval_d, eigvec_d = np.linalg.eigh(H_dense)
    eigval_s, eigvec_s = eigsh(H_sparse, k=1, which='SA')
    
    vec_d = eigvec_d[:, 0]
    vec_s = eigvec_s[:, 0]

    vec_d /= np.linalg.norm(vec_d)
    vec_s /= np.linalg.norm(vec_s)
    
    overlap = np.vdot(vec_d, vec_s)
    fidelity = np.abs(overlap)**2
    energy_match = np.abs(eigval_d[0] - eigval_s[0]) < atol

    return fidelity, energy_match

In [ ]:
state = 11
site = 3
g = 0.1

H_K_V_s = hamiltonian_sparse(state, site, g)
H_K_V_d = hamiltonian_dense(state, site, g)

print(sparse_allclose(sp.csr_matrix(H_K_V_d[0]), H_K_V_s[0]))
print(sparse_allclose(sp.csr_matrix(H_K_V_d[1]), H_K_V_s[1]))
print(sparse_allclose(sp.csr_matrix(H_K_V_d[2]), H_K_V_s[2]))

In [ ]:
state = 5
site = 5

H_K_V_sp = hamiltonian_big_sparse(state, site, g, H_K_V_s)

H_K_V_de = hamiltonian_big_dense(state, site, g, H_K_V_d)

print(sparse_allclose(sp.csr_matrix(H_K_V_de[0]), H_K_V_sp[0]))
print(sparse_allclose(sp.csr_matrix(H_K_V_de[1]), H_K_V_sp[1]))
print(sparse_allclose(sp.csr_matrix(H_K_V_de[2]), H_K_V_sp[2]))
print(np.allclose(H_K_V_de[3], H_K_V_sp[3], atol=1e-7))

This gives us the idea wether the eigenvalues and eigenvalues match for the ground states bewteen the sparse and dense.

In [ ]:
print(compare_ground_states(H_K_V_de[0], H_K_V_sp[0]))

## Testing the degenaret eigenvectors.

The sparse smallest eigenvalue diagonalization does not give consistent eigenvalues but eigenvectors in the same degenerate space as dense.

In [ ]:
def check_eigenvec(H_dense: np.ndarray, H_sparse: sp.csr_matrix):

    eigval_dense, eigvecs_dense = np.linalg.eigh(H_dense)
    eigval_sparse, eigvecs_sparse = sp.linalg.eigsh(H_sparse, k=1, which='SA')

    psi_sparse = eigvecs_sparse[:, 0]

    # Identify degeneracy threshold
    ground_energy = eigval_dense[0]
    degeneracy_tol = 1e-10  # you can adjust this
    degenerate_indices = np.where(np.abs(eigval_dense - ground_energy) < degeneracy_tol)[0]

    # Extract dense ground subspace
    ground_subspace = eigvecs_dense[:, degenerate_indices]  # shape: (dim, d)

    # Project sparse vector into dense ground subspace
    projection_coeffs = ground_subspace.conj().T @ psi_sparse  # shape: (d,)
    psi_projected = ground_subspace @ projection_coeffs        # back to shape (dim,)

    # Compute fidelity between original and projected
    fidelity_to_subspace = np.linalg.norm(projection_coeffs)**2
    print("Fidelity to dense ground state subspace:", fidelity_to_subspace)

##

In [ ]:
state = 5
site = 5

H_K_V_sparse = hamiltonian_general_sparse(state, site, g)
H_K_V_dense = hamiltonian_general_dense(state, site, g)

print(sparse_allclose(sp.csr_matrix(H_K_V_dense[0]), H_K_V_sparse[0]))
print(sparse_allclose(sp.csr_matrix(H_K_V_dense[1]), H_K_V_sparse[1]))
print(sparse_allclose(sp.csr_matrix(H_K_V_dense[2]), H_K_V_sparse[2]))

In [ ]:
check_eigenvec(H_K_V_de[0], H_K_V_sp[0])
check_eigenvec(H_K_V_dense[0], H_K_V_sparse[0])

In [ ]:
state = 7
site = 5
g = 0.1

In [ ]:
H_K_V_sparse = hamiltonian_general_sparse(state, site, g)

In [ ]:
H_K_V_s = hamiltonian_sparse(state, site, g)

In [ ]:
H_K_V_dense = hamiltonian_general_dense(state, site, g)

In [ ]:
H_K_V_d = hamiltonian_dense(state, site, g)

Stress testing hamiltonian_genreal sparse.

In [ ]:
# ---------- CONFIG ----------
max_state = 11   # must be odd; try increasing this cautiously
max_site = 61     # must be odd; try up to your system's memory
g = 1            # coupling strength

log_file = "hamiltonian_general_stress_test_log.txt"

# ---------- Logging ----------
def log(msg):
    timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_file, "a") as f:
        f.write(f"{timestamp} {msg}\n")

# ---------- Header ----------
log("\n===================== NEW STRESS TEST =====================")
log(f"Testing odd state/site values up to state={max_state}, site={max_site}\n")

last_success = None
start_time = time.time()

try:
    for site in range(3, max_site + 1, 2):  # only odd sites
        state = 11
        try:
            log(f"Trying state={state}, site={site}...")
            t0 = time.time()
            H, _, _ = hamiltonian_general_sparse(state, site, g)
            t1 = time.time()
            dim = H.shape[0]
            nnz = H.nnz if sp.issparse(H) else np.count_nonzero(H)
            log(f"✓ Success: dim={dim}, nnz={nnz}, time={t1 - t0:.2f}s")
            last_success = (state, site, dim, nnz)
        except MemoryError:
            log(f"🧨 MemoryError at state={state}, site={site}. Halting test.")
            raise
        except Exception as e:
            log(f"⚠️ Exception at state={state}, site={site}")
            log(traceback.format_exc())
            raise
except Exception:
    pass

end_time = time.time()
log(f"\n⏱️ Test completed in {end_time - start_time:.2f} seconds.")

if last_success:
    s, L, d, n = last_success
    log(f"\n✅ Last successful configuration:")
    log(f"  state = {s}")
    log(f"  site  = {L}")
    log(f"  dim   = {d}")
    log(f"  nnz   = {n}")
else:
    log("\n❌ No successful Hamiltonians generated.")

In [ ]:
state = 10000

K_p, V_p = build_V_in_p(state)

In [ ]:
K, V = build_V(state)
Vp = basis_m_to_p_matrix_conversion(V)

In [ ]:
V_p.shape

(100000000, 100000000)

# Testing standart hamiltonian.

In [32]:
state = 3

In [33]:
K, V = dn.write_matrix_elements((state-1)//2)

V = V + V.T - np.diag(np.diag(V))
V_tensor = V.reshape(state, state, state, state)  # Adjust if needed

h_full = dn.basis_m_to_p_matrix_conversion(K, state)
v_full = dn.basis_m_to_p_matrix_conversion(V_tensor, state)

In [35]:
K_sp, V_sp = build_V(state)

V_sp = basis_m_to_p_matrix_conversion(V_sp)

In [36]:
V

array([[ 0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  0. ,  1.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  1.5,  0. ],
       [ 1.5,  0. , -0.5,  0. ,  0. ,  0. , -0.5,  0. ,  1.5],
       [ 0. ,  1.5,  0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1.5,  0. , -0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  1.5,  0. ,  0. ,  0. ,  0. ]])

In [37]:
V_sp.toarray()

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75, -0.25,  0.  , -0.25,  0.75],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.75,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.25,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.75, -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

In [8]:
np.array_equal(K_sp.toarray() , K_sp_f.toarray())

True

In [9]:
np.array_equal(V_sp.toarray() , V_sp_f.toarray())

False

In [ ]:
for i in range(state-1):

    print(f"{i}, {i+1}, {i}, {i+1} --> 0.75")
    print(f"{i+1}, {i}, {i+1}, {i} --> 0.75")

    print(f"{i+1}, {i}, {i}, {i+1} --> -0.25")
    print(f"{i}, {i+1}, {i+1}, {i} --> -0.25")

    for j in range(i, state-i-1):
        print(f"{i}, {i+1}, {i+j}, {i+1+j} --> 0.75")
        print(f"{i}, {i+1}, {i+1+j}, {i+j} --> -0.25")
        print(f"{i+1}, {i}, {i+j}, {i+1+j} --> -0.25")
        print(f"{i+1}, {i}, {i+1+j}, {i+j} --> 0.75")

        print(f"{i+j}, {i+1+j}, {i+1}, {i} --> -0.25")
        print(f"{i+1+j}, {i+j}, {i+1}, {i} --> 0.75")
        print(f"{i+j}, {i+1+j}, {i}, {i+1} --> 0.75")
        print(f"{i+1+j}, {i+j}, {i}, {i+1} --> -0.25")

0, 1, 0, 1 --> 0.75
1, 0, 1, 0 --> 0.75
1, 0, 0, 1 --> -0.25
0, 1, 1, 0 --> -0.25
0, 1, 0, 1 --> 0.75
0, 1, 1, 0 --> -0.25
1, 0, 0, 1 --> -0.25
1, 0, 1, 0 --> 0.75
0, 1, 1, 0 --> -0.25
1, 0, 1, 0 --> 0.75
0, 1, 0, 1 --> 0.75
1, 0, 0, 1 --> -0.25
0, 1, 1, 2 --> 0.75
0, 1, 2, 1 --> -0.25
1, 0, 1, 2 --> -0.25
1, 0, 2, 1 --> 0.75
1, 2, 1, 0 --> -0.25
2, 1, 1, 0 --> 0.75
1, 2, 0, 1 --> 0.75
2, 1, 0, 1 --> -0.25
1, 2, 1, 2 --> 0.75
2, 1, 2, 1 --> 0.75
2, 1, 1, 2 --> -0.25
1, 2, 2, 1 --> -0.25


In [ ]:
def V_normal(d: int): 
    V = np.zeros((d**2, d**2))
    for i in range(d):
        for j in range(d):
            for k in range(d):
                for l in range(d):
                    # if k * d + l >= i * d + j:
                        V[i*d + j, k*d + l] = dn.interaction_two_body_coplanar(i, j, k, l)
    return V

In [ ]:
def V_non_zero(state: int):
    V = np.zeros((state**2, state**2))

    for x in range(state-1):
        # V[x*state + x, (x+1)*state + (x+1)] += 0.75
        # V[x*state + x, (x+1)*state + (x+1)] += 0.75

        # V[(x+1)*state + x, x*state + (x+1)] += -0.25
        # V[x*state + (x+1), (x+1)*state + x] += -0.25

        for y in range(state - x - 1):
            print(f"Start:{x}, {y}")
            print(f"{x*(state) + (x+y)}, {(x+1)*(state) + (x+1+y)}")
            V[x*(state) + (x+y), (x+1)*(state) + (x+1+y)] += 0.75
            # V[x*state + (x+1+y), (x+1)*state + (x+y)] += -0.25
            # V[(x+1)*state + (x+y), x*state + (x+1+y)] += -0.25
            # V[(x+1)*state + (x+1+y), x*state + (x+y)] += 0.75

            # V[(x+y)*state + (x+1), (x+1+y)*state + x] += -0.25
            # V[(x+1+y)*state + (x+1), (x+y)*state + x] += 0.75
            # V[(x+y)*state + x, (x+1+y)*state + (x+1)] += 0.75
            # V[(x+1+y)*state + x, (x+y)*state + (x+1)] += -0.25
    return V

In [ ]:
x = 1
y = 1
state = 3

print(f"{x*(state) + (x+y)}, {(x+1)*(state) + (x+1+y)}")

5, 9


In [ ]:
state = 3

In [ ]:
V_non_zero(state)

Start:0, 0
0, 4
Start:0, 1
1, 5
Start:1, 0
4, 8


array([[0.  , 0.  , 0.  , 0.  , 0.75, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.75, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.75],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [ ]:
np.array_equal(V_non_zero(state) , V_normal(state))

Start:0, 0
0, 4
Start:0, 1
1, 5
Start:1, 0
4, 8


False

In [ ]:
V_normal(state)

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.75,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ],
       [ 0.75,  0.  , -0.25,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.75],
       [ 0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -0.25,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.75,  0.  , -0.25,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  ,  0.  ,  0.  ]])

In [ ]:
V_non_zero(state)

0,0
0,1
0,2
1,0
1,1
2,0


IndexError: index 9 is out of bounds for axis 1 with size 9

In [ ]:
v = V_non_zero(state) 

0,0
0,1
0,2
1,0
1,1
2,0


IndexError: index 11 is out of bounds for axis 1 with size 9